# 优化模型参数

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 设置超参数

In [3]:
# Number of Epochs - 迭代数据集的次数
# Batch Size - 在参数更新之前通过网络传播的数据样本数量。
# Learning Rate - 学习率， 每 Batch/Epoch 次更新模型参数的幅度。较小的值会产生较慢的学习速度，较大的值可能会在训练过程中产生无法预料的行为。
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 优化循环 

设置完超参数后，接下来我们在一个优化循环中训练并优化我们的模型。优化循环的每次迭代叫做一个 Epoch(时期、纪元)。

每个 Epoch 由两个主要部分构成:
- 训练循环 在训练数据集上遍历，尝试收敛到最优的参数。
- 验证/测试循环 在测试数据集上遍历，以检查模型效果是否在提升。

## 损失函数 

损失函数能衡量获得的结果相对于目标值的偏离程度，我们希望在训练中能够最小化这个损失函数。我们对给定的数据样本做出预测然后和真实标签数据对比来计算损失。

常见的损失函数包括给回归任务用的 nn.MSELoss(Mean Square Error, 均方误差)、给分类任务使用的 nn.NLLLoss(Negative Log Likelihood, 负对数似然)、nn.CrossEntropyLoss(交叉熵损失函数)结合了 nn.LogSoftmax 和 nn.NLLLoss.

In [4]:
# 初始化损失函数
loss_fn = nn.CrossEntropyLoss()

## 优化器 

优化是在每一个训练步骤中调整模型参数来减小模型误差的过程。优化算法定义了这个过程应该如何进行(在这个例子中，我们使用 Stochastic Gradient Descent-即SGD，随机梯度下降)。所有优化的逻辑都被封装在 optimizer 这个对象中。

In [5]:
# 初始化优化器
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

## 完整实现 

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    # 获取训练集的大小，dataloader 是一个可以迭代的对象，dataset 包含了所有的训练数据。
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    # 将模型设置为训练模式。
    model.train()
    # 迭代 dataloader，每次迭代返回一批数据。X 是输入数据（特征），y 是目标标签。
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        # 将输入数据 X 输入到模型中，得到模型的预测值 pred。
        pred = model(X)
        # 计算预测值与真实标签 y 之间的损失。loss_fn 是损失函数，通常是如 CrossEntropyLoss 或 MSELoss。
        loss = loss_fn(pred, y)
        # 反向传播计算梯度。损失函数相对于模型参数的梯度会被计算出来。
        loss.backward()
        # 使用优化器（如 Adam 或 SGD）来更新模型的参数，基于反向传播计算出的梯度。
        optimizer.step()
        # 清空优化器中的梯度
        optimizer.zero_grad()
        # 每 100 个 batch 打印一次训练过程中的损失。
        if batch % 100 == 0:
            # 将损失张量 loss 转换为 Python 数值类型（标量）。
            # 计算当前训练的样本数量，batch + 1 是当前的 batch 索引，len(X) 是每个 batch 的样本数量。
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test_loop(dataloader, model, loss_fn):
    # 将模型设置为评估模式
    model.eval()
    # 获取测试数据集的大小。
    size = len(dataloader.dataset)
    # 获取测试集的总 batch 数量。
    num_batches = len(dataloader)
    # 初始化测试损失和正确预测的数量。
    test_loss, correct = 0, 0
    # 使用 torch.no_grad() 上下文管理器，告诉 PyTorch 在该代码块中不需要计算梯度。
    # 这样可以节省内存和计算资源，尤其是在评估阶段，因为我们不需要更新模型参数。
    with torch.no_grad():
        for X, y in dataloader:
            # 将输入 X 传入模型，得到预测结果 pred。
            pred = model(X)
            # 累加每个 batch 的损失。
            test_loss += loss_fn(pred, y).item()
            # 计算预测正确的样本数。pred.argmax(1) 获取每个样本的最大预测值对应的类索引，和真实标签 y 比较是否相同。
            # 如果相同则认为预测正确。.sum() 计算正确预测的数量，.item() 将其转换为 Python 数值。
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    # 计算平均测试损失。
    test_loss /= num_batches
    # 计算准确率（正确预测的样本数除以总样本数）。
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

**总结**

train_loop 用于训练模型，执行前向传播、损失计算、反向传播和参数更新。每 100 个 batch 打印一次当前损失。
test_loop 用于测试模型，计算模型在测试集上的损失和准确率。使用 torch.no_grad() 来避免计算梯度，从而节省计算资源。
这两个函数是模型训练和评估的常见模板。

In [8]:
# 这是 PyTorch 提供的交叉熵损失函数，常用于分类问题，尤其是多类分类。
# 它会计算模型的输出与目标标签之间的差异。
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297294  [   64/60000]
loss: 2.283904  [ 6464/60000]
loss: 2.259512  [12864/60000]
loss: 2.248751  [19264/60000]
loss: 2.231865  [25664/60000]
loss: 2.202060  [32064/60000]
loss: 2.215302  [38464/60000]
loss: 2.175219  [44864/60000]
loss: 2.172157  [51264/60000]
loss: 2.130198  [57664/60000]
Test Error: 
 Accuracy: 46.0%, Avg loss: 2.121951 

Epoch 2
-------------------------------
loss: 2.136264  [   64/60000]
loss: 2.124358  [ 6464/60000]
loss: 2.058696  [12864/60000]
loss: 2.065912  [19264/60000]
loss: 2.009481  [25664/60000]
loss: 1.956051  [32064/60000]
loss: 1.973390  [38464/60000]
loss: 1.890167  [44864/60000]
loss: 1.895499  [51264/60000]
loss: 1.804701  [57664/60000]
Test Error: 
 Accuracy: 62.7%, Avg loss: 1.806078 

Epoch 3
-------------------------------
loss: 1.847476  [   64/60000]
loss: 1.813694  [ 6464/60000]
loss: 1.692376  [12864/60000]
loss: 1.721089  [19264/60000]
loss: 1.616598  [25664/60000]
loss: 1.586091  [32064/600